In [1]:
import pandas as pd
import numpy as np

## Data importation and cleaning

In [17]:
# Parquet file is lighter than a CSV and helps to make the analysis faster.
df = pd.read_csv(r'C:\Users\mirko\Desktop\Curso Data Science DL\ProyectoEDA_Egresos_Hospitalarios_2001-2020\notebooks\Egresos_2001-2020.csv')

C:\Users\mirko\AppData\Local\Temp\ipykernel_11092\3040402395.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\mirko\Desktop\Curso Data Science DL\ProyectoEDA_Egresos_Hospitalarios_2001-2020\notebooks\Egresos_2001-2020.csv')


Basic information about the data

In [19]:
# A quick view of the data
df.head()

,Unnamed: 0,ID_PACIENTE,SEXO,EDAD_A_OS,PREVISION,PERTENENCIA_ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,DIAS_ESTADA,CONDICION_EGRESO,DIAG1,GLOSA_DIAG1,Año
0,0,NaN,Hombre,60.0,Fonasa,Privado,Clínica Familia,2.0,Muerto,C780,TUMOR MALIGNO SECUNDARIO DEL PULMÓN,2001
1,1,NaN,Hombre,74.0,Fonasa,Publico,"Hospital Del Salvador (Santiago, Providencia)",58.0,Muerto,E145,"DIABETES MELLITUS NO ESPECIFICADA, CON COMPLIC...",2001
2,2,NaN,Hombre,71.0,Fonasa,Publico,"Hospital Del Salvador (Santiago, Providencia)",12.0,Muerto,J189,"NEUMONIA, NO ESPECIFICADA",2001
3,3,NaN,Hombre,2.0,Fonasa,Publico,Instituto de Neurocirugía Dr. Alfonso Asenjo,1.0,Muerto,S065,HEMORRAGIA SUBDURAL TRAUMÁTICA,2001
4,4,NaN,Hombre,81.0,Fonasa,Publico,Hospital Dr. Leonardo Guzmán (Antofagasta),7.0,Vivo,K830,COLANGITIS,2001


In [21]:
# the datatype of each column was established in the notebook "Data_Importation"
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26340655 entries, 0 to 26340654
Data columns (total 12 columns):
 #   Column                             Dtype  
---  ------                             -----  
 0   Unnamed: 0                         int64  
 1   ID_PACIENTE                        object 
 2   SEXO                               object 
 3   EDAD_A_OS                          float64
 4   PREVISION                          object 
 5   PERTENENCIA_ESTABLECIMIENTO_SALUD  object 
 6   GLOSA_ESTABLECIMIENTO_SALUD        object 
 7   DIAS_ESTADA                        float64
 8   CONDICION_EGRESO                   object 
 9   DIAG1                              object 
 10  GLOSA_DIAG1                        object 
 11  Año                                int64  
dtypes: float64(2), int64(2), object(8)
memory usage: 2.4+ GB


Missing data in "ID_PACIENTE" column

In [23]:
# All missing data was deleted on the importation process, except on "GLOSA_ESTABLECIMIENTO_SALUD" and "ID_PACIENTE" columns
df.isnull().sum()

Unnamed: 0                                 0
ID_PACIENTE                          1944331
SEXO                                       0
EDAD_A_OS                                  0
PREVISION                                  0
PERTENENCIA_ESTABLECIMIENTO_SALUD          0
GLOSA_ESTABLECIMIENTO_SALUD             2753
DIAS_ESTADA                                0
CONDICION_EGRESO                           0
DIAG1                                      0
GLOSA_DIAG1                                0
Año                                        0
dtype: int64

In [25]:
df.shape

(26340655, 12)

In [ ]:
df['ID'].isna().mean()

Fixing missing or incorrect values (outilers, duplicates)

In [27]:
df["ID_PACIENTE"][df["ID_PACIENTE"].duplicated(keep=False)]

0                                                NaN
1                                                NaN
2                                                NaN
3                                                NaN
4                                                NaN
                              ...                   
26340649    9F742AA5C59C96EF9B07F3EDBCF55F8DA744EE02
26340650    18C7CA0C9694DF3E3011E270C912D11B1BF2C257
26340651    18C7CA0C9694DF3E3011E270C912D11B1BF2C257
26340652    BB8EE4EAF297C191FFB91B1DADB935D6AEC58CE3
26340653    44A39983F8C088BBB8EFA4B103865EE1948CFD1A
Name: ID_PACIENTE, Length: 21314957, dtype: object

## Exploratory analysis and visualizations with matplotlib and seaborn

## Preguntas que surjan

1. ¿Cual es la tendencia en la evolución de la prevalencia de los problemas de salud a lo largo de los años?
2. dfsdf
3. dsf
4. sdf
5. sd

   